In [4]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import scipy

import collections
import datetime
import time

import geopandas as gpd

import numpy as np
 
from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from sklearn import linear_model
import statsmodels.api as sm

from linearmodels import PanelOLS, FamaMacBeth
from scipy import stats

import itertools


## Grab Data

In [8]:
os.getcwd()

'/Users/brianreed/Documents/supplyChain/extremes/extremesAnalysisCode'

In [23]:
# goodsData = pd.read_csv("../goodsData_suppliers.csv").drop(columns = {'Unnamed: 0'})
# goodsData = pd.read_csv("../goodsData_igData.csv").drop(columns = {'Unnamed: 0'})
goodsData = pd.read_csv("../utilitiesData_igData.csv").drop(columns = {'Unnamed: 0'})

# goodsData = pd.read_csv("../goodsData.csv").drop(columns = {'Unnamed: 0'})
goodsData = goodsData[~goodsData.revenueChange.isna() & 
                     ~goodsData.costChange.isna()]

# goodsData['precip5Days_annualquant_1x5Yrs'] = 1*(goodsData['precip5Days_annualquant_1x5Yrs'] > 0)

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# goodsData

Let's grab a list with all the relevant variable names. Just keep the core names and we'll grab the lags as we go through it.

Look at the full combinations of variables and whatnot, if we'd like.

In [24]:
'''
print(goodsData.columns[0:50])

cutoffVars  = ['0.95','1xQtr','1xYr','1x5Qtrs','1x5Yrs'] # ,'1x10Qtrs','1x10Yrs'
weatherVars = ['precip_','temp_','precip5Days_','temp5Days_']
statVars    = ['annualquant_','zipquant_','zipQuarterquant_']

outcomeVars = ['lnCost','lnRev','lnCostNormd','lnRevNormd']
# ['incomeChange', 'revenueChange', 'costChange'],'lnCost','lnInc','lnRev']
# ['lnCost','lnRev','lnCostNormd','lnRevNormd']


for weatherVar in weatherVars:
    for statVar in statVars:
        for cutoffVar in cutoffVars:
            indVar = weatherVar + statVar + cutoffVar
            
            for outcomeVar in outcomeVars:
                print(indVar,"~", outcomeVar)'''

'\nprint(goodsData.columns[0:50])\n\ncutoffVars  = [\'0.95\',\'1xQtr\',\'1xYr\',\'1x5Qtrs\',\'1x5Yrs\'] # ,\'1x10Qtrs\',\'1x10Yrs\'\nweatherVars = [\'precip_\',\'temp_\',\'precip5Days_\',\'temp5Days_\']\nstatVars    = [\'annualquant_\',\'zipquant_\',\'zipQuarterquant_\']\n\noutcomeVars = [\'lnCost\',\'lnRev\',\'lnCostNormd\',\'lnRevNormd\']\n# [\'incomeChange\', \'revenueChange\', \'costChange\'],\'lnCost\',\'lnInc\',\'lnRev\']\n# [\'lnCost\',\'lnRev\',\'lnCostNormd\',\'lnRevNormd\']\n\n\nfor weatherVar in weatherVars:\n    for statVar in statVars:\n        for cutoffVar in cutoffVars:\n            indVar = weatherVar + statVar + cutoffVar\n            \n            for outcomeVar in outcomeVars:\n                print(indVar,"~", outcomeVar)'

Focus on the 5 day variables now (we'll later need to edit the loops above to account for this as well, but we can do this for now.) Make them 0/1.

# Direct Effects
Look at the effects on the suppliers when they're affected directly.

The below gives us the full, clustered standard errors.

In [26]:
cutoffVarsYr = ['0.95']  #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_'] #, 'temp5Days_', 'precip5Days_'] # , 'precip_']#, , ] #[,]
statVarsYr   = ['zipquant_', 'ffquant_', 'zipQuarterquant_', 'indQuarterquant_'] #  , , ]  #,'zipQuarterquant_']
outcomeVars  = ['lnRevNormd', 'lnCostNormd']# , 'lnRev', 'lnCost', 'revenueChange', 'costChange']

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna()] # & ~goodsData.lnCostNormd.isna()]


start = time.time()

results = pd.DataFrame()

i = 0
for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:
        for statVar in statVarsYr:                     
            for cutoffVar in cutoffVarsYr:
                i = i + 1
                indVar = weatherVar + statVar + cutoffVar
                
                
                print(outcomeVar, "~", indVar)


                # find: concurrent ; or lagged supplier data
                X = goodsData.loc[:,((goodsData.columns.str.contains(indVar)) | 
                                                (goodsData.columns.str.contains('indQtr_')) |
                                                (goodsData.columns.str.contains('gvkey_')))]#  | 
                                                # (goodsData.columns.str.contains('firmQtr_')) |
                                                #(goodsData.columns.str.contains('ageQtr_')) |
                                                #(goodsData.columns.str.contains('sizeQtr_')) |
                                                #(goodsData.columns.str.contains('profitQtr_')))]
                
                
                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]
                
                
                model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                
                # results.loc[i,'ind'] = ind


                results.loc[i,'outcomeVar'] = outcomeVar
                results.loc[i,'weatherVar'] = weatherVar
                results.loc[i,'statVar']    = statVar
                results.loc[i,'cutoffVar']  = cutoffVar
                # results.loc[i,'firmVar']    = firmVar


                results.loc[i,'lag0']       = coeff[0]
                results.loc[i,'lag1']       = coeff[1]
                results.loc[i,'lag2']       = coeff[2]
                results.loc[i,'lag3']       = coeff[3]
                results.loc[i,'lag4']       = coeff[4]

                results.loc[i,'pval0']      = pvals[0]
                results.loc[i,'pval1']      = pvals[1]
                results.loc[i,'pval2']      = pvals[2]
                results.loc[i,'pval3']      = pvals[3]
                results.loc[i,'pval4']      = pvals[4]

                
                results.to_csv("../../data/utilitiesResults_rightInds_noCtrls.csv")
                
                print( time.time() - start)

lnRevNormd ~ precip_zipquant_0.95
0.9773828983306885
lnRevNormd ~ precip_ffquant_0.95
2.520321846008301
lnRevNormd ~ precip_zipQuarterquant_0.95
3.8356997966766357
lnRevNormd ~ precip_indQuarterquant_0.95
5.075920820236206
lnRevNormd ~ temp_zipquant_0.95
6.713193893432617
lnRevNormd ~ temp_ffquant_0.95
7.9631028175354
lnRevNormd ~ temp_zipQuarterquant_0.95
9.13026475906372
lnRevNormd ~ temp_indQuarterquant_0.95
10.570092916488647
lnCostNormd ~ precip_zipquant_0.95
11.576021909713745
lnCostNormd ~ precip_ffquant_0.95
12.521055936813354
lnCostNormd ~ precip_zipQuarterquant_0.95
13.596194982528687
lnCostNormd ~ precip_indQuarterquant_0.95
14.867467880249023
lnCostNormd ~ temp_zipquant_0.95
15.853435754776001
lnCostNormd ~ temp_ffquant_0.95
16.845340728759766
lnCostNormd ~ temp_zipQuarterquant_0.95
17.825721740722656
lnCostNormd ~ temp_indQuarterquant_0.95
19.082683801651


In [15]:
print(ind)

35


In [54]:
results.to_csv("../../data/utilitiesResults_rightInds.csv")

## Industry-Specific
Go through every famafrench industry and run the regressions above.

In [29]:
cutoffVarsYr = ['0.95']  #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_'] #, 'temp5Days_', 'precip5Days_'] # , 'precip_']#, , ] #[,]
statVarsYr   = ['zipquant_', 'ffquant_', 'zipQuarterquant_', 'indQuarterquant_'] #  , , ]  #,'zipQuarterquant_']
outcomeVars  = ['lnRevNormd', 'lnCostNormd'] # , 'lnRev', 'lnCost', 'revenueChange', 'costChange']

industries = range(1,44)

start = time.time()

# results = pd.DataFrame()

# i = 0
for ind in industries:
    print('##########################################################')
    print(ind)
    filename = '../../data/companyData/igData_ind' + str(ind) + '.csv'           
    goodsData = pd.read_csv(filename).drop(columns = {'Unnamed: 0'})

    
    for outcomeVar in outcomeVars:
        for weatherVar in weatherVars:
            for statVar in statVarsYr:                     
                for cutoffVar in cutoffVarsYr:
                    
                    i = i + 1
                    
                    
                    
                    goodsData = goodsData[~goodsData.lnRev.isna() & 
                                         ~goodsData.lnCost.isna()]
                    
                    
                    indVar = weatherVar + statVar + cutoffVar


                    print(outcomeVar, "~", indVar)


                    # find: concurrent ; or lagged supplier data
                    X = goodsData.loc[:,((goodsData.columns.str.contains(indVar)) | 
                                                    (goodsData.columns.str.contains('indQtr_')) |
                                                    (goodsData.columns.str.contains('gvkey_')))]#   | 
                                                    # (goodsData.columns.str.contains('ageQtr_')) |
                                                    # (goodsData.columns.str.contains('sizeQtr_')) |
                                                    # (goodsData.columns.str.contains('profitQtr_')))]


                    firms = goodsData['gvkey']


                    y = goodsData[outcomeVar]


                    model = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                    pvals = model.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]
                    coeff =  model.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)])]

                    # results.loc[i,'ind'] = ind

                    results.loc[i,'industry'] = ind

                    results.loc[i,'outcomeVar'] = outcomeVar
                    results.loc[i,'weatherVar'] = weatherVar
                    results.loc[i,'statVar']    = statVar
                    results.loc[i,'cutoffVar']  = cutoffVar
                    # results.loc[i,'firmVar']    = firmVar


                    results.loc[i,'lag0']       = coeff[0]
                    results.loc[i,'lag1']       = coeff[1]
                    results.loc[i,'lag2']       = coeff[2]
                    results.loc[i,'lag3']       = coeff[3]
                    results.loc[i,'lag4']       = coeff[4]

                    results.loc[i,'pval0']      = pvals[0]
                    results.loc[i,'pval1']      = pvals[1]
                    results.loc[i,'pval2']      = pvals[2]
                    results.loc[i,'pval3']      = pvals[3]
                    results.loc[i,'pval4']      = pvals[4]


                    results.to_csv("../../results_byInds_controls.csv")

                    print( time.time() - start)

##########################################################
1
lnRevNormd ~ precip_zipquant_0.95
0.07257604598999023
lnRevNormd ~ precip_ffquant_0.95
0.09779596328735352
lnRevNormd ~ precip_zipQuarterquant_0.95
0.1230769157409668
lnRevNormd ~ precip_indQuarterquant_0.95
0.14989018440246582
lnRevNormd ~ temp_zipquant_0.95
0.17541098594665527
lnRevNormd ~ temp_ffquant_0.95
0.20368599891662598
lnRevNormd ~ temp_zipQuarterquant_0.95
0.23078608512878418
lnRevNormd ~ temp_indQuarterquant_0.95
0.25897812843322754
lnCostNormd ~ precip_zipquant_0.95
0.2842409610748291
lnCostNormd ~ precip_ffquant_0.95
0.3092811107635498
lnCostNormd ~ precip_zipQuarterquant_0.95
0.33768701553344727
lnCostNormd ~ precip_indQuarterquant_0.95
0.3616199493408203
lnCostNormd ~ temp_zipquant_0.95
0.38814496994018555
lnCostNormd ~ temp_ffquant_0.95
0.4130561351776123
lnCostNormd ~ temp_zipQuarterquant_0.95
0.4394040107727051
lnCostNormd ~ temp_indQuarterquant_0.95
0.467120885848999
#######################################

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


0.7540879249572754
lnRevNormd ~ precip_ffquant_0.95
0.885915994644165
lnRevNormd ~ precip_zipQuarterquant_0.95
1.0629069805145264
lnRevNormd ~ precip_indQuarterquant_0.95
1.3033630847930908
lnRevNormd ~ temp_zipquant_0.95
1.4354901313781738
lnRevNormd ~ temp_ffquant_0.95
1.6433491706848145
lnRevNormd ~ temp_zipQuarterquant_0.95
1.831162929534912
lnRevNormd ~ temp_indQuarterquant_0.95
1.9615869522094727
lnCostNormd ~ precip_zipquant_0.95
2.120281934738159
lnCostNormd ~ precip_ffquant_0.95
2.2785730361938477
lnCostNormd ~ precip_zipQuarterquant_0.95
2.452705144882202
lnCostNormd ~ precip_indQuarterquant_0.95
2.6366422176361084
lnCostNormd ~ temp_zipquant_0.95
2.7686421871185303
lnCostNormd ~ temp_ffquant_0.95
2.9263648986816406
lnCostNormd ~ temp_zipQuarterquant_0.95
3.109281063079834
lnCostNormd ~ temp_indQuarterquant_0.95
3.2855958938598633
##########################################################
3
lnRevNormd ~ precip_zipquant_0.95
3.363555908203125
lnRevNormd ~ precip_ffquant_0.95
3

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


6.560106992721558
lnRevNormd ~ precip_ffquant_0.95
6.70330810546875
lnRevNormd ~ precip_zipQuarterquant_0.95
6.8689069747924805
lnRevNormd ~ precip_indQuarterquant_0.95
7.016474962234497
lnRevNormd ~ temp_zipquant_0.95
7.219398021697998
lnRevNormd ~ temp_ffquant_0.95
7.372079849243164
lnRevNormd ~ temp_zipQuarterquant_0.95
7.5281171798706055
lnRevNormd ~ temp_indQuarterquant_0.95
7.707603931427002
lnCostNormd ~ precip_zipquant_0.95
7.8573899269104
lnCostNormd ~ precip_ffquant_0.95
8.013055086135864
lnCostNormd ~ precip_zipQuarterquant_0.95
8.16641902923584
lnCostNormd ~ precip_indQuarterquant_0.95
8.318401098251343
lnCostNormd ~ temp_zipquant_0.95
8.48801589012146
lnCostNormd ~ temp_ffquant_0.95
8.671525955200195
lnCostNormd ~ temp_zipQuarterquant_0.95
8.843641996383667
lnCostNormd ~ temp_indQuarterquant_0.95
9.023848056793213
##########################################################
8
lnRevNormd ~ precip_zipquant_0.95
9.114528179168701
lnRevNormd ~ precip_ffquant_0.95
9.1568551063537

...

300.6525340080261
lnRevNormd ~ precip_indQuarterquant_0.95
300.71874499320984
lnRevNormd ~ temp_zipquant_0.95
300.7832028865814
lnRevNormd ~ temp_ffquant_0.95
300.84265899658203
lnRevNormd ~ temp_zipQuarterquant_0.95
300.9076051712036
lnRevNormd ~ temp_indQuarterquant_0.95
300.9796199798584
lnCostNormd ~ precip_zipquant_0.95
301.0422558784485
lnCostNormd ~ precip_ffquant_0.95
301.1067337989807
lnCostNormd ~ precip_zipQuarterquant_0.95
301.186331987381
lnCostNormd ~ precip_indQuarterquant_0.95
301.2465970516205
lnCostNormd ~ temp_zipquant_0.95
301.317645072937
lnCostNormd ~ temp_ffquant_0.95
301.43737411499023
lnCostNormd ~ temp_zipQuarterquant_0.95
301.51238107681274
lnCostNormd ~ temp_indQuarterquant_0.95
301.5726869106293
##########################################################
40
lnRevNormd ~ precip_zipquant_0.95
301.6598069667816
lnRevNormd ~ precip_ffquant_0.95
301.70719289779663
lnRevNormd ~ precip_zipQuarterquant_0.95
301.76022601127625
lnRevNormd ~ precip_indQuarterquant_0.95

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ precip_zipquant_0.95
302.8099160194397
lnRevNormd ~ precip_ffquant_0.95
303.0835909843445
lnRevNormd ~ precip_zipQuarterquant_0.95
303.4041681289673
lnRevNormd ~ precip_indQuarterquant_0.95
303.70044016838074
lnRevNormd ~ temp_zipquant_0.95
303.96561312675476
lnRevNormd ~ temp_ffquant_0.95
304.2749300003052
lnRevNormd ~ temp_zipQuarterquant_0.95
304.58374309539795
lnRevNormd ~ temp_indQuarterquant_0.95
304.84830594062805
lnCostNormd ~ precip_zipquant_0.95
305.1170299053192
lnCostNormd ~ precip_ffquant_0.95
305.37921500205994
lnCostNormd ~ precip_zipQuarterquant_0.95
305.7362058162689
lnCostNormd ~ precip_indQuarterquant_0.95
306.02494597435
lnCostNormd ~ temp_zipquant_0.95
306.3006842136383
lnCostNormd ~ temp_ffquant_0.95
306.6016731262207
lnCostNormd ~ temp_zipQuarterquant_0.95
306.91111612319946
lnCostNormd ~ temp_indQuarterquant_0.95
307.18395805358887
##########################################################
42


/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


lnRevNormd ~ precip_zipquant_0.95
307.8737459182739
lnRevNormd ~ precip_ffquant_0.95
308.31308603286743
lnRevNormd ~ precip_zipQuarterquant_0.95
308.77740716934204
lnRevNormd ~ precip_indQuarterquant_0.95
309.25381302833557
lnRevNormd ~ temp_zipquant_0.95
309.684543132782
lnRevNormd ~ temp_ffquant_0.95
310.0998420715332
lnRevNormd ~ temp_zipQuarterquant_0.95
310.5275001525879
lnRevNormd ~ temp_indQuarterquant_0.95
310.9836299419403
lnCostNormd ~ precip_zipquant_0.95
311.3750422000885
lnCostNormd ~ precip_ffquant_0.95
311.80173110961914
lnCostNormd ~ precip_zipQuarterquant_0.95
312.23445296287537
lnCostNormd ~ precip_indQuarterquant_0.95
312.7019531726837
lnCostNormd ~ temp_zipquant_0.95
313.1231451034546
lnCostNormd ~ temp_ffquant_0.95
313.50830602645874
lnCostNormd ~ temp_zipQuarterquant_0.95
313.9746949672699
lnCostNormd ~ temp_indQuarterquant_0.95
314.4150538444519
##########################################################
43
lnRevNormd ~ precip_zipquant_0.95
314.68492698669434
lnRe

# Faster and More Heuristic

The below gives us unclustered standard errors, output to a csv file.

In [ ]:
def findSE(X,reg,y):
    N = len(X)
    p = len(X.columns) + 1  # plus one because LinearRegression adds an intercept term

    X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
    X_with_intercept[:, 0] = 1
    X_with_intercept[:, 1:p] = X.values

    y_hat = reg.predict(X)
    residuals = y.values - y_hat
    residual_sum_of_squares = residuals.T @ residuals
    sigma_squared_hat = residual_sum_of_squares / (N - p)
    var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_squared_hat

    se0 = var_beta_hat[1, 1] ** 0.5
    se1 = var_beta_hat[2, 2] ** 0.5
    se2 = var_beta_hat[3, 3] ** 0.5
    se3 = var_beta_hat[4, 4] ** 0.5
    se4 = var_beta_hat[5, 5] ** 0.5
    se5 = var_beta_hat[6, 6] ** 0.5
    '''se6 = var_beta_hat[7, 7] ** 0.5
    se7 = var_beta_hat[8, 8] ** 0.5
    se8 = var_beta_hat[9, 9] ** 0.5'''
    return([abs(reg.coef_[0]/se0),abs(reg.coef_[1]/se1),abs(reg.coef_[2]/se2),
            abs(reg.coef_[3]/se3),abs(reg.coef_[4]/se4),abs(reg.coef_[5]/se5)]
          )

'''        
abs(reg.coef_[0]/se0),
          abs(reg.coef_[1]/se1),
          abs(reg.coef_[2]/se2),
          abs(reg.coef_[3]/se3),
          abs(reg.coef_[4]/se4),
          abs(reg.coef_[5]/se5),
          "SE0: ", se0,
          "SE1: ", se1,
          "SE2: ", se2,
          "SE3: ", se3,
          "SE4: ", se4,
          "SE5: ", se5,

'''


In [ ]:
goodsData.columns[0:75]

In [ ]:
goodsData.columns[0:50]

In [ ]:
inds = [1, 2, 6, 7, 18, 31, 41, 42]

for ind in inds:

    tempData = goodsData[goodsData.famafrench == ind]
    print(tempData.shape)


In [ ]:
'''cutoffVarsYr = ['0.95'] # ,'1xYr']                                    #,'1x5Yrs'] #, ] # ,'1xQtr', '1x5Qtrs'
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['zipquant_','zipQuarterquant_']
outcomeVars  = ['lnRev', 'revenueChange'] # ,'lnCost',  'costChange'] # [,'lnRevNormd','lnCostNormd'] # 'revenueChange' 'costChange',
firmVars     = ['firmQtr_'] # 'gvkey'
'''

# try this by industry
cutoffVarsYr = ['0.95'] # ,'1xYr']                                    #,'1x5Yrs'] #, ] # ,'1xQtr', '1x5Qtrs'
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['ffquant_','indQuarterquant_']
outcomeVars  = ['lnRev', 'revenueChange',  'lnCost',  'costChange'] # [,'lnRevNormd','lnCostNormd'] # 'revenueChange' 'costChange',
firmVars     = ['firmQtr_']


inds = [1, 2, 6, 7, 18, 31, 41, 42]

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() &
                      ~goodsData.lnCostNormd.isna() & ~goodsData.lnRevNormd.isna()]

start = time.time()

results = pd.DataFrame()
i = 0
for ind in inds:
    print('#######################################################################################',ind)
    for outcomeVar in outcomeVars:
        for weatherVar in weatherVars:
            for statVar in statVarsYr:                     
                for cutoffVar in cutoffVarsYr:
                    for firmVar in firmVars:
                        tempData = goodsData[goodsData.famafrench == ind]
                        
                        i = i + 1
                        indVar = weatherVar + statVar + cutoffVar


                        print(outcomeVar, "~", indVar, "|", firmVar)


                        # find: concurrent ; or lagged supplier data
                        X = tempData.loc[:,((tempData.columns.str.contains(indVar)) |
                                          (tempData.columns.str.contains('indQtr_')) |
                                          # (goodsData.columns.str.contains('gvkey_'))) |   # &   
                                          # (goodsData.columns.str.contains('firmQtr_'))) |
                                          (tempData.columns.str.contains(firmVar)))] # |
                        '''(tempData.columns.str.contains('ageQtr_')) |
                          (tempData.columns.str.contains('sizeQtr_')) |
                          (tempData.columns.str.contains('profitQtr_'))]   #  & '''

                                          # (goodsData.columns.str.contains('firmQtr_')))       & 
                                        # ~(goodsData.columns.str.contains('lag4')) &
                                                                        # ~(goodsData.columns.str.contains('lag2')) & 


                        X = X[X.columns[(X.sum(axis = 0) >= 4)]]
                        # print(X.columns)
                        firms = tempData['gvkey']


                        y = tempData[outcomeVar]


                        ######################################
                        # fit the model on this subset
                        reg = linear_model.LinearRegression()
                        reg.fit(X,y)


                        # print('Coeff: ' , reg.coef_[0:5], 'SE type (looking >2): ', findSE(X,reg,y))
                        results.loc[i,'ind'] = ind


                        results.loc[i,'outcomeVar'] = outcomeVar
                        results.loc[i,'weatherVar'] = weatherVar
                        results.loc[i,'statVar']    = statVar
                        results.loc[i,'cutoffVar']  = cutoffVar
                        results.loc[i,'firmVar']    = firmVar


                        results.loc[i,'lag0']       = reg.coef_[0]
                        results.loc[i,'lag1']       = reg.coef_[1]
                        results.loc[i,'lag2']       = reg.coef_[2]
                        results.loc[i,'lag3']       = reg.coef_[3]
                        results.loc[i,'lag4']       = reg.coef_[4]



                        seratios = findSE(X,reg,y)

                        results.loc[i,'ratio0']       = seratios[0]
                        results.loc[i,'ratio1']       = seratios[1]
                        results.loc[i,'ratio2']       = seratios[2]
                        results.loc[i,'ratio3']       = seratios[3]
                        results.loc[i,'ratio4']       = seratios[4]

                        # print(results)

                        print(time.time() - start)

                        print('*******************************************************************')
                    
results.to_csv("../../data/results_notNormd.csv")

In [ ]:
i = 0

results = pd.DataFrame()
results.loc[i,'outcome'] = 0
results

# Indirect Effects
This is almost exactly the same but with supplier information in place of the direct company information.

In [ ]:
cutoffVarsYr = ['1xYr'] #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['annualquant_']  #,'zipQuarterquant_']
outcomeVars  = ['costChange', 'revenueChange', 'lnRevNormd', 'lnCostNormd'] # ['revenueChange'] #[, 'costChange']#,'lnCost','lnInc','lnRev']

goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() & ~goodsData.lnCostNormd.isna()]
goodsData['scTercile']  = pd.qcut(goodsData['suppliers'], 3, labels=False, duplicates = 'drop')


start = time.time()

for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:
        for statVar in statVarsYr:                     
            for cutoffVar in cutoffVarsYr:
                indVar = weatherVar + statVar + cutoffVar
                
                
                print(outcomeVar, "~", indVar)


                # find: concurrent ; or lagged supplier data
                X = goodsData.loc[:,(((goodsData.columns.str.contains(indVar)) & goodsData.columns.str.contains('supplier')) | 
                                                (goodsData.columns.str.contains('indQtr_')) |
                                                (goodsData.columns.str.contains('gvkey_'))  | 
                                                (goodsData.columns.str.contains('ageQtr_')) |
                                                (goodsData.columns.str.contains('sizeQtr_')) |
                                                (goodsData.columns.str.contains('profitQtr_')) | 
                                                (goodsData.columns == 'suppliers')) &   
                                                # (goodsData.columns.str.contains('firmQtr_')))       & 
                                                ~(goodsData.columns.str.contains('gvkey_1013')) & # do 1 minus the FEs
                                                ~(goodsData.columns.str.contains('indQtr_12010_1'))]# & 
                                                # ~(goodsData.columns.str.contains('lag4'))]     
                
                
                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]
                
                
                results = sm.OLS(y, X).fit(cov_type='cluster',cov_kwds={'groups': firms},use_t=True)
                pvals = results.pvalues[0:len(goodsData.columns[goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('supplier_')])]
                coeff = results.params[0:len(goodsData.columns[goodsData.columns.str.contains(indVar)  & goodsData.columns.str.contains('supplier_')])]
                print(coeff)
                print(pvals)
                
                
                print( time.time() - start)
                
                
                

In [ ]:
cutoffVarsYr = ['1xYr'] #, ] # ,'1xQtr''1x5Qtrs',
weatherVars  = ['precip_', 'temp_', 'precip5Days_', 'temp5Days_'] #[,]
statVarsYr   = ['annualquant_']  #,'zipQuarterquant_']
outcomeVars  = ['revenueChange',  'costChange'] # ['revenueChange'] #['incomeChange', 'revenueChange', 'costChange']#,'lnCost','lnInc','lnRev']


# 'revenueChange',  'costChange', 
# 'lnRevNormd', 'lnCostNormd'


goodsData = goodsData[~goodsData.lnRev.isna() & ~goodsData.lnCost.isna() & ~goodsData.lnCostNormd.isna()]


start = time.time()

for outcomeVar in outcomeVars:
    for weatherVar in weatherVars:
        for statVar in statVarsYr:                     
            for cutoffVar in cutoffVarsYr:
                indVar = weatherVar + statVar + cutoffVar
                supplierIndVar = 'supplier_' + weatherVar + statVar + cutoffVar
                
                
                print(outcomeVar, "~", indVar, supplierIndVar)


                # find: concurrent ; or lagged supplier data
                X = goodsData.loc[:,((goodsData.columns.str.contains(indVar) & goodsData.columns.str.contains('supplier')) |
                                  (goodsData.columns.str.contains('indQtr_'))       |
                                  (goodsData.columns.str.contains('gvkey_'))) &   
                                # (goodsData.columns.str.contains('firmQtr_')))       & 
                                  ~(goodsData.columns.str.contains('gvkey_1045'))     & # do 1 minus the FEs
                                ~(goodsData.columns.str.contains('indQtr_12013_1'))]#  &
                                # ~(goodsData.columns.str.contains('lag4')) &
                                # (goodsData.columns.str.contains('ageQtr_')) |
                                # (goodsData.columns.str.contains('sizeQtr_')) |
                                # (goodsData.columns.str.contains('profitQtr_'))) & 
                                # ~(goodsData.columns.str.contains('firmQtr_10451'))] # & # do 1 minus the FEs
                                # ~(goodsData.columns.str.contains('indQtr_12013_1'))]# & 
                                # ~(goodsData.columns.str.contains('lag2')) & 
                                 
                                
                print(X.columns[0:5])

                X = X[X.columns[(X.sum(axis = 0) >= 4)]]
                firms = goodsData['gvkey']


                y = goodsData[outcomeVar]


                ######################################
                # fit the model on this subset
                reg = linear_model.LinearRegression()
                reg.fit(X,y)


                print('Coeff: ' , reg.coef_[0:5], 'SE type (looking >2): ', findSE(X,reg,y))
                
                print(time.time() - start)
                
                print('*******************************************************************')
                


# Quarterly-Specific Effects
Look at what is going on within a quarter, for a specific zipcode. 

Note: this is effectively deprecated at this point because the zip-level effects mask the differences in the underlying weather distributions, ie some regions would likely benefit from getting warmer. If we want to do a zip-level study, we should also include some sort of threshold here.

Do this for all qtrs.

In [ ]:
cutoffVarsQtr = ['0.95','1xQtr','1x5Qtrs']# ,'1x5Qtrs'] # ,
weatherVars   = ['precip5Days_'] # ['precip5Days_','temp5Days_']
statVarQtr    = 'zipQuarterquant_'
outcomeVars   = ['revenueChange'] #['incomeChange', 'revenueChange', 'costChange']



for weatherVar in weatherVars:
    for cutoffVar in cutoffVarsQtr:
        indVar = weatherVar + statVarQtr + cutoffVar            
        for outcomeVar in outcomeVars:
            print(outcomeVar, "~", indVar)



            X = goodsData.loc[:,(goodsData.columns == indVar) | 
                              (goodsData.columns.str.contains('indQtr_')) |
                              (goodsData.columns.str.contains('gvkey_')) & 
                             ~(goodsData.columns.str.contains('gvkey_1004')) & # do 1 minus the FEs
                             ~(goodsData.columns.str.contains('indQtr_422019_4'))]


            X = X[X.columns[(X.sum(axis = 0) >= 4)]]
            firms = goodsData['gvkey']


            y = goodsData[outcomeVar]


            ######################################
            # fit the model on this subset
            reg = linear_model.LinearRegression()
            reg.fit(X,y)


            print('Coeff: ' , reg.coef_[0:5], 'SE type (looking >2): ', findSE(X,reg,y))
            print(time.time() - start)

In [ ]:
X[X.columns[(X.sum(axis = 0) >= 4)]].shape